In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/30-days-of-ml/sample_submission.csv
/kaggle/input/30-days-of-ml/train.csv
/kaggle/input/30-days-of-ml/test.csv


In [2]:
import pandas as pd
train=pd.read_csv(r"../input/30-days-of-ml/train.csv")
test=pd.read_csv(r"../input/30-days-of-ml/test.csv")

In [3]:
train.describe()

,id,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target
count,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000
mean,250018.576947,0.527335,0.460926,0.490498,0.496689,0.491654,0.510526,0.467476,0.537119,0.498456,0.474872,0.474492,0.473216,0.494561,0.508273,8.241979
std,144450.150010,0.230599,0.214003,0.253346,0.219199,0.240074,0.228232,0.210331,0.218140,0.239920,0.218007,0.255949,0.222022,0.247292,0.222950,0.746555
min,1.000000,-0.118039,-0.069309,-0.056104,0.130676,0.255908,0.045915,-0.224689,0.203763,-0.260275,0.117896,0.048732,0.052608,-0.074208,0.151050,0.140329
25%,124772.500000,0.405965,0.310494,0.300604,0.329783,0.284188,0.354141,0.342873,0.355825,0.332486,0.306874,0.276017,0.308151,0.289074,0.300669,7.742071
50%,250002.500000,0.497053,0.427903,0.502462,0.465026,0.390470,0.488865,0.429383,0.504661,0.439151,0.434620,0.459975,0.433812,0.422887,0.472400,8.191373
75%,375226.500000,0.668060,0.615113,0.647512,0.664451,0.696599,0.669625,0.573383,0.703441,0.606056,0.614333,0.691579,0.642057,0.714502,0.758447,8.728634
max,499999.000000,1.058443,0.887253,1.034704,1.039560,1.055424,1.067649,1.111552,1.032837,1.040229,0.982922,1.055960,1.071444,0.975035,0.905992,10.411992


In [4]:
train.isnull().sum().sum()

0

In [5]:
train.dtypes.unique()

array([dtype('int64'), dtype('O'), dtype('float64')], dtype=object)

In [6]:
x=train.drop(["target"],axis=1)
y=train["target"]

In [7]:
cat_col=list(x.select_dtypes(include=["object"]).columns)
num_col=list(x.select_dtypes(exclude=["object"]).columns)

In [8]:
from sklearn.preprocessing import OneHotEncoder
oe=OneHotEncoder()
from sklearn.impute import SimpleImputer
se=SimpleImputer(strategy="most_frequent")

In [9]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Normalizer
nz=Normalizer()
cpipe=Pipeline(steps=[("impute",se),("encode",oe),("normalizer",nz)])
npipe=Pipeline(steps=[("impute",SimpleImputer(strategy="mean")),("normalize",nz)])

In [10]:
from sklearn.compose import ColumnTransformer 
transform=ColumnTransformer(transformers=[("cat",cpipe,cat_col),("num",npipe,num_col)])

In [11]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(x,y,train_size=0.8)
from sklearn.metrics import mean_absolute_error

In [12]:
def processing(model,transform=transform,xtrain=xtrain,ytrain=ytrain,xtest=xtest,ytest=ytest):
    out=Pipeline(steps=[("transform",transform),("model",model)])
    out.fit(xtrain,ytrain)
    pred=out.predict(xtest)
    return mean_absolute_error(pred,ytest),pred

In [13]:
test

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,...,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13
0,0,B,B,B,C,B,B,A,E,E,...,0.476739,0.376350,0.337884,0.321832,0.445212,0.290258,0.244476,0.087914,0.301831,0.845702
1,5,A,B,A,C,B,C,A,E,C,...,0.285509,0.860046,0.798712,0.835961,0.391657,0.288276,0.549568,0.905097,0.850684,0.693940
2,15,B,A,A,A,B,B,A,E,D,...,0.697272,0.683600,0.404089,0.879379,0.275549,0.427871,0.491667,0.384315,0.376689,0.508099
3,16,B,B,A,C,B,D,A,E,A,...,0.719306,0.777890,0.730954,0.644315,1.024017,0.391090,0.988340,0.411828,0.393585,0.461372
4,17,B,B,A,C,B,C,A,E,C,...,0.313032,0.431007,0.390992,0.408874,0.447887,0.390253,0.648932,0.385935,0.370401,0.900412
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,499987,B,A,A,C,B,D,A,E,E,...,0.287454,0.543800,0.682378,1.028978,1.022741,0.683903,0.877273,0.532410,0.605397,0.884581
199996,499990,B,A,A,C,B,B,A,E,C,...,0.794881,0.432778,0.389775,0.359871,0.550013,0.492082,0.202295,0.416875,0.406205,0.758665
199997,499991,A,B,B,C,B,B,A,E,C,...,0.514487,0.060997,0.171741,0.317185,0.150340,0.122109,0.390524,0.334026,0.378987,0.839416
199998,499994,A,A,A,C,B,D,A,D,A,...,0.286144,1.061710,0.819811,0.901241,0.555339,0.844315,0.894193,0.794102,0.844279,0.890473


In [14]:
from xgboost import XGBRegressor
model=XGBRegressor()

In [15]:
pipe2=Pipeline(steps=[("transform",transform),("model",model)])
pipe2.fit(xtrain,ytrain)


Pipeline(steps=[('transform',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encode',
                                                                   OneHotEncoder()),
                                                                  ('normalizer',
                                                                   Normalizer())]),
                                                  ['cat0', 'cat1', 'cat2',
                                                   'cat3', 'cat4', 'cat5',
                                                   'cat6', 'cat7', 'cat8',
                                                   'cat9']),
                                                 ('num',
                                                  Pipelin

In [16]:
pred=pipe2.predict(test)

In [17]:
id=test["id"]
df=pd.DataFrame({"id":id,"target":pred})
df.to_csv("output.csv",index=False)

In [18]:
catt_col=list(train.select_dtypes(include=["object"]).columns)
numt_col=list(train.select_dtypes(exclude=["object"]).columns)

In [19]:
from sklearn.linear_model import LinearRegression
model=LinearRegression()
processing(model)

(0.5914784109370869,
 array([8.24336624, 8.27582169, 8.23074341, ..., 8.32540131, 8.25491333,
        8.33679199]))

In [ ]:
from sklearn.ensemble import RandomForestRegressor
model=RandomForestRegressor()
processing(model)

In [ ]:
from sklearn.tree import DecisionTreeRegressor
model=DecisionTreeRegressor()
processing(model)

In [ ]:
from xgboost import XGBRegressor
model=XGBRegressor()
processing(model)

In [ ]:
XGBRegressor

In [ ]:
from sklearn.model_selection import cross_val_score
model=DecisionTreeRegressor()
cross_val_score()